<h2 align = "center">Sprawozdanie mofit</h2>
<h4 align="center">Łukasz Puchała, Karol Grzywa</h4>

In [1]:
import numpy as np
from numba import njit
import math
import matplotlib.pyplot as plt
from math import sin, cos, pi, e, exp
from scipy.linalg import eigh
import Utilis

m = 0.067 # m0
h = 1
at2meV = 27211.6
ab = 0.05292
j = 0 + 1j
at2s = 2.42e-5*1e-12

L = 100/ab
N = 2
a = L/(2*N)

In [2]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')
wezly = np.array(grid_N2_L100)
wezly 

array([[  1. , -50. , -50. ],
       [  2. , -25. , -50. ],
       [  3. ,   0. , -50. ],
       [  4. ,  25. , -50. ],
       [  5. , -50. , -25. ],
       [  6. , -25. , -25. ],
       [  7. ,   0. , -25. ],
       [  8. ,  25. , -25. ],
       [  9. , -50. ,   0. ],
       [ 10. , -25. ,   0. ],
       [ 11. ,   0. ,   0. ],
       [ 12. ,  25. ,   0. ],
       [ 13. , -50. ,  25. ],
       [ 14. , -25. ,  25. ],
       [ 15. ,   0. ,  25. ],
       [ 16. ,  25. ,  25. ],
       [ 17. ,  50. , -50. ],
       [ 18. ,  50. , -25. ],
       [ 19. ,  50. ,   0. ],
       [ 20. ,  50. ,  25. ],
       [ 21. , -50. ,  50. ],
       [ 22. , -25. ,  50. ],
       [ 23. ,   0. ,  50. ],
       [ 24. ,  25. ,  50. ],
       [ 25. ,  50. ,  50. ],
       [ 26. , -37.5, -50. ],
       [ 27. , -25. , -37.5],
       [ 28. , -50. , -37.5],
       [ 29. , -37.5, -25. ],
       [ 30. , -37.5, -37.5],
       [ 31. , -12.5, -50. ],
       [ 32. ,   0. , -37.5],
       [ 33. , -12.5, -25. ],
       [ 3

In [3]:
nlg_N2_l100 = Utilis.ReadFile('Files/nlg_N_2_L_100.dat.txt')
nlg = np.array(nlg_N2_l100)
nlg

array([[ 1,  1,  1],
       [ 1,  2,  2],
       [ 1,  3,  5],
       [ 1,  4,  6],
       [ 1,  5, 26],
       [ 1,  6, 27],
       [ 1,  7, 28],
       [ 1,  8, 29],
       [ 1,  9, 30],
       [ 2,  1,  2],
       [ 2,  2,  3],
       [ 2,  3,  6],
       [ 2,  4,  7],
       [ 2,  5, 31],
       [ 2,  6, 32],
       [ 2,  7, 27],
       [ 2,  8, 33],
       [ 2,  9, 34],
       [ 3,  1,  3],
       [ 3,  2,  4],
       [ 3,  3,  7],
       [ 3,  4,  8],
       [ 3,  5, 35],
       [ 3,  6, 36],
       [ 3,  7, 32],
       [ 3,  8, 37],
       [ 3,  9, 38],
       [ 4,  1,  4],
       [ 4,  2, 17],
       [ 4,  3,  8],
       [ 4,  4, 18],
       [ 4,  5, 39],
       [ 4,  6, 40],
       [ 4,  7, 36],
       [ 4,  8, 41],
       [ 4,  9, 42],
       [ 5,  1,  5],
       [ 5,  2,  6],
       [ 5,  3,  9],
       [ 5,  4, 10],
       [ 5,  5, 29],
       [ 5,  6, 43],
       [ 5,  7, 44],
       [ 5,  8, 45],
       [ 5,  9, 46],
       [ 6,  1,  6],
       [ 6,  2,  7],
       [ 6,  

In [11]:
# funkcja, co bierze poz. wezla naroznego elementu k o loklanym nr i
print( Utilis.xy_nlg(2, 5, nlg, wezly) )
# wspóółrzędne rzeczywiste punktu 
print( Utilis.xryr(k= 1, ksi1= 1, ksi2= -1, nlg= nlg, wezly= wezly) )

(-12.5, -50.0)
(-25.0, -50.0)
